In [1]:

!pip install datasets

from datasets import load_dataset, Dataset


!pip install pandas

import pandas as pd

!pip install transformers
!pip install torch

import torch
from transformers import BertTokenizer, BertModel



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1

In [ ]:
# selecting data from mexican border conflict year
mexican_border_data = load_dataset("dell-research-harvard/AmericanStories",
    "subset_years",
    year_list=["1916"]
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for dell-research-harvard/AmericanStories contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dell-research-harvard/AmericanStories
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the 

Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1916': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1916.tar.gz'}


In [ ]:
#this is how you access the article content for a specific row in the data
print(mexican_border_data['1916'][0]['article'])


In [ ]:
#using bert-case-uncased for the model to allow for capitalized and uncapitalized words to be treated the same
#tokenizer converts articles into tokens
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model takes the tokens and makes embeddings
model = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
#converting to a data frame
mexican_border_df = pd.DataFrame.from_dict(mexican_border_data['1916'])



In [ ]:
#droppping unneeded rows
mexican_border_df.drop(['newspaper_name','edition','page','headline','byline'], axis=1)

In [ ]:
#function for encoding the article
def encode_article(article):
    inputs = tokenizer(article, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)


mexican_border_df['encoded_article'] = mexican_border_df['article'].apply(lambda x: encode_article(x).detach().numpy())



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

racist_keywords = {
    'Mexican': ['greaser', 'wetback', 'beaner'],
    'German': ['kraut', 'hun', 'boche'],
    'Russian': ['commie', 'red', 'ruskie'],
    'Japanese': ['jap', 'nip', 'yellow'],
    'Korean': ['gook', 'slope', 'zipperhead']
}

# Encode the racist keywords
encoded_keywords = {ethnicity: [encode_article(word).detach().numpy() for word in words] for ethnicity, words in racist_keywords.items()}


In [ ]:
#calculates similarity between article embeddings and keyword embeddings
def calculate_similarity(article_embedding, keywords_embeddings):
    similarities = [cosine_similarity(article_embedding, keyword_embedding) for keyword_embedding in keywords_embeddings]
    return max(similarities)

#determines if an article is racist by seeing if it meets a certain threshold of racist language
def is_racist(article_embedding, ethnicity):
    keywords_embeddings = encoded_keywords[ethnicity]
    similarity_threshold = 0.7  # Define a threshold for similarity
    max_similarity = calculate_similarity(article_embedding, keywords_embeddings)
    return 1 if max_similarity > similarity_threshold else 0

# Label each article for each conflict ethnicity
mexican_border_df['Mexican_racist'] = mexican_border_df['encoded_article'].apply(lambda x: is_racist(x, 'Mexican'))

#find average number of racist articles in 1916
mexican_border_df['Mexican_racist'].mean()

#find average number of racists articles in a year when no mexican border conflict

#compare the two averages

#repeat the process with the mexican border with the four other conflicts

#see if there is a trend with all of the different conflicts
